In [1]:
import sys

sys.path.append('../GSL')

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import pickle

from torch_geometric.data import Data as gData
from torch_geometric.utils import to_networkx, to_undirected
from torch_geometric.nn import MessagePassing

import matplotlib.pyplot as plt

from torch_geometric_temporal.nn.recurrent import DCRNN

/Users/juhyeonkim/opt/anaconda3/envs/GSL/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torch_geometric.utils import to_undirected, to_dense_adj, add_self_loops, sort_edge_index, remove_self_loops, is_undirected

# GTS

In [4]:
from models.GTS.gts_graph_learning2 import GTS_Graph_Learning2
from models.GTS.gts_forecasting_module import GTS_Forecasting_Module
from models.GTS.self_attention_graph_learning import Attention_Graph_Learning
from models.GTS.DCRNN import DCRNN
from utils.utils import build_fully_connected_edge_idx, build_batch_edge_index
from dataset.make_traffic_dataset import TrafficDatasetLoader
from torch_geometric.utils import dense_to_sparse, to_dense_adj
from models.MTGNN.mtgnn_graph_learning import MTGNN_Graph_Learning

from utils.adjacency_matrix_sampling import gumbel_softmax_structure_sampling, weight_matrix_construct, top_k_structure_construct 


from glob import glob
import yaml
from easydict import EasyDict as edict

In [5]:
config_file = glob('../GSL/config/GTS/*.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [6]:
config.dataset.root = '../GSL/data/PEMS-BAY'

In [7]:
loader = TrafficDatasetLoader(raw_data_dir=config.dataset.root, dataset_name=config.dataset.name)
dataset,entire_inputs = loader.get_dataset(num_timesteps_in=config.encoder_step,
                                                 num_timesteps_out=config.decoder_step,
                                                 batch_size=config.train.batch_size)

In [8]:
entire_inputs.shape

torch.Size([325, 2, 52105])

In [9]:
config.graph_learning.sampling = 'None'

In [10]:
edge_index = build_fully_connected_edge_idx(num_nodes=config.nodes_num)

In [11]:
edge_index.shape

torch.Size([2, 105625])

In [12]:
gl = GTS_Graph_Learning2(config)

In [13]:
al = Attention_Graph_Learning(config)
ml = MTGNN_Graph_Learning(config)

In [14]:
gts_style = gl(entire_inputs, edge_index)

In [15]:
attention = al(entire_inputs)
low_rank_approx = ml()

In [16]:
gts_style.shape

torch.Size([105625, 1])

In [17]:
attention.shape

torch.Size([325, 325])

In [18]:
low_rank_approx.shape

torch.Size([325, 325])

batch, adj = gumbel_softmax_structure_sampling(gts_style, edge_index, 2, 0.3, 325, False)


In [32]:
batch, adj = top_k_structure_construct(low_rank_approx, 2, 50, 325, False)

In [33]:
batch

tensor([[  0,   0,   0,  ..., 649, 649, 649],
        [  1,   8,  27,  ..., 645, 647, 648]])

In [39]:
_, batch_weight, weight = weight_matrix_construct(attention, 2, 325, False)

In [40]:
batch_weight.shape

torch.Size([211250, 1])

In [41]:
weight.shape

torch.Size([325, 325])

In [42]:
batch_weight[:325*325]

tensor([[-0.0181],
        [ 0.0122],
        [ 0.0031],
        ...,
        [ 0.0198],
        [ 0.0699],
        [-0.1066]], grad_fn=<SliceBackward0>)

In [43]:
weight

tensor([[-0.0181,  0.0122,  0.0031,  ...,  0.0202,  0.0039,  0.0164],
        [-0.0069,  0.0239, -0.0229,  ...,  0.0048,  0.0314, -0.0238],
        [-0.0323, -0.0132, -0.0075,  ...,  0.0297, -0.0266,  0.0066],
        ...,
        [ 0.0029, -0.0019, -0.0023,  ...,  0.0059, -0.0003, -0.0065],
        [-0.0057,  0.0027,  0.0016,  ..., -0.0065,  0.0042, -0.0076],
        [-0.0010,  0.0628, -0.0755,  ...,  0.0198,  0.0699, -0.1066]],
       grad_fn=<AddmmBackward0>)